In [ ]:
import pandas as pd
input_path = '../tmp-data/ss_range_bars.1'
df = pd.read_csv(f'{input_path}.csv', parse_dates=['timestamp'], index_col='timestamp')
df

In [ ]:
def mark_close_to_bollinger(df, threshold=0.1, window=3):
    """
    Adds a new column to the DataFrame indicating whether the Close value is within
    `threshold` fraction of the Bollinger Bands (bb_upper or bb_lower) in a window of
    `window` rows forward and backward.
    """
    for i, row in df.iterrows():
        # find the indices of nearby rows
        start = max(0, i - window)
        end = min(len(df), i + window + 1)
        
        # check if any of the nearby rows are close to a bollinger band
        close_to_band = (
            ((df.loc[start:i, 'Close'] >= df.loc[i, 'bb_lower'] * (1 - threshold)) & 
             (df.loc[start:i, 'Close'] <= df.loc[i, 'bb_lower'] * (1 + threshold))) |
            
            ((df.loc[i:end, 'Close'] >= df.loc[i, 'bb_upper'] * (1 - threshold)) & 
             (df.loc[i:end, 'Close'] <= df.loc[i, 'bb_upper'] * (1 + threshold)))
        ).any()
        
        # assign the result to a new column
        df.at[i, 'close_to_bollinger'] = int(close_to_band)

        # multiply the window size by the frequency of the index
        freq = pd.infer_freq(df.index)
        window_timedelta = pd.Timedelta(window * freq)
        
        # add the timedelta to the index to get the value for the new column
        df.at[i, 'window_end'] = df.index[min(i + window, len(df) - 1)] + window_timedelta
        df.at[i, 'window_start'] = df.index[max(i - window, 0)] - window_timedelta

# apply the function to the DataFrame
mark_close_to_bollinger(df)
df